<a href="https://colab.research.google.com/github/GRTO/DiplomadoPucp/blob/master/Stochastic_Universal_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Author:** Gerson Toribio Ossio

In [0]:
import sys
import time
import random
import numpy as np
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
import heapq

# Stochastic Universal Sampling Algorithm

In [0]:
def makeWheel(population):
    wheel = []
    total = sum(p.fitness for p in population)
    top = 0
    for p in population:
        f = p.fitness/total
        wheel.append((top, top+f, p.chromosome, p.fitness))
        top += f
    return wheel

def binSearch(wheel, num):
    mid = len(wheel)//2
    low, high, answer, fitness = wheel[mid]
    if low<=num<=high:
        return (answer, fitness)
    elif low < num:
        return binSearch(wheel[mid+1:], num)
    else:
        return binSearch(wheel[:mid], num)

def select(wheel, N):
    stepSize = 1.0/N
    answer = []
    r = random.random()
    answer.append(binSearch(wheel, r))
    while len(answer) < N:
        r += stepSize
        if r>1:
            r %= 1
        answer.append(binSearch(wheel, r))
    return answer

# Class creation

In [0]:
class Individual_binary(object):   
   
    def __init__(self, chromosome, allele_pool):
            self.chromosome = chromosome[:]
            self.allele_pool = allele_pool
            self.fitness = -1  # -1 indica que el individuo no ha sido evaluado
            
    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre individuos self y other "
        c = random.randrange(len(self.chromosome))
        ind1 = Individual(self.chromosome[:c] + other.chromosome[c:], allele_pool)
        ind2 = Individual(other.chromosome[:c] + self.chromosome[c:], allele_pool)
        return [ind1, ind2]   
    
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual(chromosome1, allele_pool)
        ind2 = Individual(chromosome2, allele_pool)
        return [ind1, ind2] 

    def mutate_position(self): 
        """       Bit flip
        Cambia aleatoriamente un alelo de un gen."""
        mutated_chromosome = deepcopy(self.chromosome)
        mutGene = random.randrange(0,len(mutated_chromosome)) 
        newAllele = allele_pool[random.randrange(0,len(allele_pool))]
        mutated_chromosome[mutGene] = newAllele
        return Individual(mutated_chromosome, allele_pool)
        
    def mutate_swap(self):
        "Escoge dos genes e intercambia sus alelos"
        mutated_chromosome = deepcopy(self.chromosome)
        mutGen1 = random.randrange(0,len(mutated_chromosome))
        mutGen2 = random.randrange(0,len(mutated_chromosome))
        temp = mutated_chromosome[mutGen1]
        mutated_chromosome[mutGen1] = mutated_chromosome[mutGen2]
        mutated_chromosome[mutGen2] = temp
        return Individual(mutated_chromosome, allele_pool)

In [0]:
class Individual_Float(object):   
   
    def __init__(self, chromosome):
            self.chromosome = chromosome[:]
            self.fitness = -1  # -1 indica que el individuo no ha sido evaluado
             
    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre individuos self y other "
        c = random.randrange(len(self.chromosome))
        ind1 = Individual(self.chromosome[:c] + other.chromosome[c:])
        ind2 = Individual(other.chromosome[:c] + self.chromosome[c:])
        return [ind1, ind2]   

    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual(chromosome1)
        ind2 = Individual(chromosome2)
        return [ind1, ind2] 
  

    def mutate_position(self): 
        """       Bit flip
        Cambia aleatoriamente un alelo de un gen."""
        mutated_chromosome = deepcopy(self.chromosome)
        mutGene = random.randrange(0,len(mutated_chromosome)) 
        newAllele = random.randrange(0,len(mutated_chromosome))
        mutated_chromosome[mutGene] = newAllele
        return Individual(mutated_chromosome)    
        
    def mutate_swap(self):
        "Escoge dos genes e intercambia sus alelos"
        mutated_chromosome = deepcopy(self.chromosome[:])
        mutGen1 = random.randrange(0,len(mutated_chromosome))
        mutGen2 = random.randrange(0,len(mutated_chromosome))
        temp = mutated_chromosome[mutGen1]
        mutated_chromosome[mutGen1] = mutated_chromosome[mutGen2]
        mutated_chromosome[mutGen2] = temp
        return Individual(mutated_chromosome)

In [0]:
class Individual_permutation(object):   
   
    def __init__(self, chromosome):
            self.chromosome = chromosome[:]
            self.fitness = -1  # -1 indica que el individuo no ha sido evaluado
     
    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre individuos self y other "
        c = random.randrange(len(self.chromosome))
        ind1 = Individual(self.chromosome[:c] + other.chromosome[c:])
        ind2 = Individual(other.chromosome[:c] + self.chromosome[c:])
        return [ind1, ind2]   
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other"
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual(chromosome1)
        ind2 = Individual(chromosome2)
        return [ind1, ind2]     
    
    def mutate_position(self): 
        """       Bit flip
        Cambia aleatoriamente un alelo de un gen."""
        mutated_chromosome = deepcopy(self.chromosome)
        mutGene = random.randrange(0,len(mutated_chromosome)) 
        newAllele = random.randrange(0,len(mutated_chromosome))
        mutated_chromosome[mutGene] = newAllele
        return Individual(mutated_chromosome)    
        
    def mutate_swap(self):
        "Escoge dos genes e intercambia sus alelos"
        mutated_chromosome = deepcopy(self.chromosome[:])
        mutGen1 = random.randrange(0,len(mutated_chromosome))
        mutGen2 = random.randrange(0,len(mutated_chromosome))
        temp = mutated_chromosome[mutGen1]
        mutated_chromosome[mutGen1] = mutated_chromosome[mutGen2]
        mutated_chromosome[mutGen2] = temp
        return Individual(mutated_chromosome)

In [0]:
class Individual_string:   
    
    def __init__(self, chromosome, allele_pool):  # el constructor recibe el cromosoma  y el pool de alelos posibles
        self.chromosome = chromosome[:]
        self.allele_pool = allele_pool
        self.fitness = -1  # -1 indica que el individuo no ha sido evaluado
        
    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre individuos self y other "
        c = random.randrange(len(self.chromosome))
        ind1 = Individual(self.chromosome[:c] + other.chromosome[c:], allele_pool)
        ind2 = Individual(other.chromosome[:c] + self.chromosome[c:], allele_pool)
        return [ind1, ind2]   
    
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual(chromosome1, allele_pool)
        ind2 = Individual(chromosome2, allele_pool)
        return [ind1, ind2] 

    def mutate_position(self):
        "Cambia aleatoriamente el alelo de un gen."
        mutated_chromosome = deepcopy(self.chromosome)
        mutGene = random.randrange(0,len(mutated_chromosome)) 
        newAllele = allele_pool[random.randrange(0,len(allele_pool))]
        mutated_chromosome[mutGene] = newAllele
        return Individual(mutated_chromosome, allele_pool)
        
    def mutate_swap(self):
        "Escoge dos genes e intercambia sus alelos"
        mutated_chromosome = deepcopy(self.chromosome)
        mutGen1 = random.randrange(0,len(mutated_chromosome))
        mutGen2 = random.randrange(0,len(mutated_chromosome))
        temp = mutated_chromosome[mutGen1]
        mutated_chromosome[mutGen1] = mutated_chromosome[mutGen2]
        mutated_chromosome[mutGen2] = temp
        return Individual(mutated_chromosome, allele_pool)

In [0]:
def display(population):
    listaAG=[]
    for i in range(len(population)):
        listaAG.append([population[i].chromosome,population[i].fitness])

    data=pd.DataFrame(listaAG)
    data.columns = ['Poblacion','fitness']
    return data

In [0]:
def display_string(population):
    listaAG=[]
    for i in range(len(population)):
        listaAG.append([''.join(population[i].chromosome),population[i].fitness])

    data=pd.DataFrame(listaAG)
    data.columns = ['Poblacion','fitness']
    return data

# Binary

In [0]:
def init_population_binary(pop_number, chromosome_size, allele_pool):
    
    num_alleles = len(allele_pool)
    population = []
    for i in range(pop_number):
        new_chromosome = [allele_pool[random.randrange(0, num_alleles)] for j in range(chromosome_size)]
        population.append( Individual_binary(new_chromosome, allele_pool) )
    return population

In [0]:
num_individuals=7
ind_size=4

# Inicializa una poblacion inicial de forma aleatoria
# construye el pool de alelos con los caracteres 
allele_pool = [0,1]
population = init_population_binary(num_individuals,ind_size, allele_pool)

display(population) #Imprime la primera poblacion

,Poblacion,fitness
0,"[1, 0, 0, 0]",-1
1,"[1, 1, 0, 0]",-1
2,"[0, 0, 0, 1]",-1
3,"[0, 1, 1, 1]",-1
4,"[0, 0, 1, 1]",-1
5,"[0, 1, 0, 1]",-1
6,"[0, 0, 0, 0]",-1


In [0]:
population[0].fitness=2
population[1].fitness=23
population[2].fitness=11
population[3].fitness=17
population[4].fitness=22
population[5].fitness=5
population[6].fitness=19
display(population)

,Poblacion,fitness
0,"[1, 0, 0, 0]",2
1,"[1, 1, 0, 0]",23
2,"[0, 0, 0, 1]",11
3,"[0, 1, 1, 1]",17
4,"[0, 0, 1, 1]",22
5,"[0, 1, 0, 1]",5
6,"[0, 0, 0, 0]",19


**Run wheel**

In [0]:
wheel = makeWheel(population)
select(wheel, 4)

[([0, 1, 1, 1], 17),
 ([0, 0, 1, 1], 22),
 ([0, 0, 0, 0], 19),
 ([1, 1, 0, 0], 23)]

# Float

In [0]:
def init_population_float(pop_number, rst):
    population = []
    state_length=2
    ## Crea la poblacion inicial con cromosomas aleatorios
    for i in range(pop_number):
        new_chromosome = [2*rst*random.uniform(0, 1)-rst for j in range(state_length)]
        random.shuffle(new_chromosome)
        population.append(Individual_Float(new_chromosome))
    return population

In [0]:
rst = 5.12  # Para restringir la poblacion entre [-rst,rst] 
num_individuals=15

# Inicializa una poblacion inicial de forma aleatoria
population = init_population_float(num_individuals, rst)
display(population)

,Poblacion,fitness
0,"[0.15967176639681924, -1.264455474444842]",-1
1,"[-1.993620241960656, 2.9663471355587996]",-1
2,"[0.2874610744443622, -3.625149592073626]",-1
3,"[-3.0939409718657296, -1.6330491841549408]",-1
4,"[-0.08425673683951906, -1.5071868801635424]",-1
5,"[-3.1480285540924138, 1.8093629784204515]",-1
6,"[-2.2310353170805035, -1.9650933078405477]",-1
7,"[2.9370116388130905, 0.796468160742406]",-1
8,"[1.0776665303792106, 2.3096550439476218]",-1
9,"[-1.8942015792946187, -5.089880921922644]",-1


In [0]:
population[0].fitness=2
population[1].fitness=23
population[2].fitness=11
population[3].fitness=17
population[4].fitness=22
population[5].fitness=5
population[6].fitness=7
population[7].fitness=12
population[8].fitness=29
population[9].fitness=3
population[10].fitness=15
population[11].fitness=19
population[12].fitness=6
population[13].fitness=25
population[14].fitness=8
display(population)

,Poblacion,fitness
0,"[0.15967176639681924, -1.264455474444842]",2
1,"[-1.993620241960656, 2.9663471355587996]",23
2,"[0.2874610744443622, -3.625149592073626]",11
3,"[-3.0939409718657296, -1.6330491841549408]",17
4,"[-0.08425673683951906, -1.5071868801635424]",22
5,"[-3.1480285540924138, 1.8093629784204515]",5
6,"[-2.2310353170805035, -1.9650933078405477]",7
7,"[2.9370116388130905, 0.796468160742406]",12
8,"[1.0776665303792106, 2.3096550439476218]",29
9,"[-1.8942015792946187, -5.089880921922644]",3


**Run wheel**

In [0]:
wheel = makeWheel(population)
select(wheel, 5)

[([2.8244287071560557, 4.99323204132197], 15),
 ([-2.9314744009253038, -3.5276082912050652], 25),
 ([-1.993620241960656, 2.9663471355587996], 23),
 ([-3.0939409718657296, -1.6330491841549408], 17),
 ([2.9370116388130905, 0.796468160742406], 12)]

# Permutación

In [0]:
def init_population_permutation(pop_number, size_chromosoma):
    population = []
    state_length=2
    ## Crea la poblacion inicial con cromosomas aleatorios
    for i in range(pop_number):
        new_chromosome = [j for j in range(1,size_chromosoma+1)]
        random.shuffle(new_chromosome)
        population.append(Individual_permutation(new_chromosome))
    return population

In [0]:
size_chromosoma = 14
num_individuals=15

# Inicializa una poblacion inicial de forma aleatoria
population = init_population_permutation(num_individuals, size_chromosoma)
display(population)

,Poblacion,fitness
0,"[3, 8, 1, 10, 12, 7, 11, 2, 14, 4, 5, 6, 13, 9]",-1
1,"[7, 2, 12, 4, 13, 10, 14, 3, 5, 6, 9, 1, 8, 11]",-1
2,"[7, 6, 10, 1, 14, 2, 11, 12, 13, 5, 3, 4, 8, 9]",-1
3,"[3, 5, 14, 12, 1, 7, 9, 13, 2, 11, 6, 10, 4, 8]",-1
4,"[13, 7, 4, 6, 2, 14, 10, 9, 11, 1, 3, 12, 5, 8]",-1
5,"[10, 8, 2, 5, 4, 1, 11, 7, 12, 6, 9, 3, 13, 14]",-1
6,"[14, 10, 11, 2, 12, 13, 8, 1, 7, 9, 3, 4, 6, 5]",-1
7,"[12, 11, 5, 6, 2, 1, 7, 10, 3, 8, 14, 9, 4, 13]",-1
8,"[2, 8, 10, 3, 5, 6, 1, 7, 9, 11, 12, 4, 13, 14]",-1
9,"[6, 4, 14, 3, 8, 11, 1, 7, 13, 10, 2, 5, 9, 12]",-1


In [0]:
population[0].fitness=2
population[1].fitness=23
population[2].fitness=11
population[3].fitness=17
population[4].fitness=22
population[5].fitness=5
population[6].fitness=7
population[7].fitness=12
population[8].fitness=28
population[9].fitness=3
population[10].fitness=15
population[11].fitness=9
population[12].fitness=6
population[13].fitness=25
population[14].fitness=18
display(population)

,Poblacion,fitness
0,"[3, 8, 1, 10, 12, 7, 11, 2, 14, 4, 5, 6, 13, 9]",2
1,"[7, 2, 12, 4, 13, 10, 14, 3, 5, 6, 9, 1, 8, 11]",23
2,"[7, 6, 10, 1, 14, 2, 11, 12, 13, 5, 3, 4, 8, 9]",11
3,"[3, 5, 14, 12, 1, 7, 9, 13, 2, 11, 6, 10, 4, 8]",17
4,"[13, 7, 4, 6, 2, 14, 10, 9, 11, 1, 3, 12, 5, 8]",22
5,"[10, 8, 2, 5, 4, 1, 11, 7, 12, 6, 9, 3, 13, 14]",5
6,"[14, 10, 11, 2, 12, 13, 8, 1, 7, 9, 3, 4, 6, 5]",7
7,"[12, 11, 5, 6, 2, 1, 7, 10, 3, 8, 14, 9, 4, 13]",12
8,"[2, 8, 10, 3, 5, 6, 1, 7, 9, 11, 12, 4, 13, 14]",28
9,"[6, 4, 14, 3, 8, 11, 1, 7, 13, 10, 2, 5, 9, 12]",3


**Run wheel**

In [0]:
wheel = makeWheel(population)
select(wheel, 4)

[([2, 8, 10, 3, 5, 6, 1, 7, 9, 11, 12, 4, 13, 14], 28),
 ([9, 6, 5, 14, 12, 13, 11, 10, 4, 2, 3, 7, 8, 1], 6),
 ([7, 2, 12, 4, 13, 10, 14, 3, 5, 6, 9, 1, 8, 11], 23),
 ([13, 7, 4, 6, 2, 14, 10, 9, 11, 1, 3, 12, 5, 8], 22)]

# String

In [0]:
def init_population_string(pop_number, chromosome_size, allele_pool):
    
    num_alleles = len(allele_pool)
    population = []
    for i in range(pop_number):
        new_chromosome = [allele_pool[random.randrange(0, num_alleles)] for j in range(chromosome_size)]
        population.append( Individual_string(new_chromosome, allele_pool) )
    return population

In [0]:
target_string = 'Inteligencia Artificial en la PUCP'

# construye el pool de alelos con los caracteres en mayuscula, minuscula y espacio en blanco
allele_pool = []
allele_pool.extend( [chr(x) for x in range(65, 91)] )   # caracteres ASCII en mayuscula
allele_pool.extend( [chr(x) for x in range(97, 123)] )  # caracteres ASCII en minuscula
allele_pool.extend(' ')  # espacio en blanco

# Inicializa una poblacion inicial de forma aleatoria
num_individuals = 7
population = init_population_string(num_individuals, len(target_string), allele_pool)
display_string(population)

,Poblacion,fitness
0,jKvtdmuggVlOxIjozVSWISLTiKzYFFdnfI,-1
1,cfSVIjxtkoChffEaGEISKNpbwAEqnKxEfN,-1
2,wjQxCGnCZIRfyjawmrZXiJjxebiWoiyVwc,-1
3,VBvxlkHwlpdZIXybdyUkWEgqaKMiAAkAr,-1
4,YOBiwe iZjsHQPMifwewbRy R dKsBHiwz,-1
5,KTmBBjDZ bhmzRshCUJeCHLRETKRtDYFXM,-1
6,kOjmNVCjdYdVOTyetAAVnUGtANaBFYFsBg,-1


In [0]:
population[0].fitness=2
population[1].fitness=23
population[2].fitness=11
population[3].fitness=17
population[4].fitness=22
population[5].fitness=5
population[6].fitness=19
display_string(population)

,Poblacion,fitness
0,jKvtdmuggVlOxIjozVSWISLTiKzYFFdnfI,2
1,cfSVIjxtkoChffEaGEISKNpbwAEqnKxEfN,23
2,wjQxCGnCZIRfyjawmrZXiJjxebiWoiyVwc,11
3,VBvxlkHwlpdZIXybdyUkWEgqaKMiAAkAr,17
4,YOBiwe iZjsHQPMifwewbRy R dKsBHiwz,22
5,KTmBBjDZ bhmzRshCUJeCHLRETKRtDYFXM,5
6,kOjmNVCjdYdVOTyetAAVnUGtANaBFYFsBg,19


In [0]:
def showString(values):
  finalValue=[]
  for e in values:
    lst = list(e)
    lst[0] = ''.join(lst[0])
    e = tuple(lst)
    finalValue.append(e)
  return finalValue

**Run wheel**

In [0]:
wheel = makeWheel(population)
showString(select(wheel, 3))

[('wjQxCGnCZIRfyjawmrZXiJjxebiWoiyVwc', 11),
 ('YOBiwe iZjsHQPMifwewbRy R dKsBHiwz', 22),
 ('jKvtdmuggVlOxIjozVSWISLTiKzYFFdnfI', 2)]